In [ ]:
import numpy as np
import pickle
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import utils

reg = pickle.load(open('LR.pkl', 'rb'))

# Elo Theory

The basic idea is that a team $X$ has a rating $R_X$. These ratings can help us calculate the expected result that a given team, say $A$, will beat another team, say $B$:

$ \begin{align}
    \nonumber \bar{E}[A \text{ beats } B] &= \bar{E}_{A, B} \\
    \nonumber &= \frac{1}{1 + 10^{(R_B - R_A) / 400}}
\end{align} $

Now, this formula is actually an estimation of the expected value. We can not know the actual value and that is why we need the ratings in the first place. As such, we need to update the ratings (and their inherent probabilities) everytime we actually observe a match between $A$ and $B$. Assume that we observed said match, and let us denote the score outcome from the viewpoint of $A$ as $S_A$. $S_A$ will be $1$ if $A$ won, and $0$ if $A$ lost and $1/2$ if they tied. Similarly, from the viewpoint of $B$, $S_B$ is $1$ if $B$ won, $0$ if they lost and $1/2$ if they tied. Let us observe what happens when we express the expectation as its original probabillistic definition:

$ \begin{align}
    \nonumber \bar{E}_{A, B} = 1 \cdot \bar{P}(A \text{ beats } B) + 1/2 \cdot \bar{P}(A \text{ ties with } B) + 0 \cdot \bar{P}(A \text{ loses to } B)
\end{align} $

We can't know each individal probability in this context. However, if our game has no ties, we can simplify this expression to:

$ \begin{align}
    \nonumber \bar{E}_{A, B} &= \bar{P}(A \text{ beats } B) \\
    \nonumber &= \bar{P}_{A, B}
\end{align} $

This means that, for games with only win/lose results, the Elo system actually approximates win probabilities. This interpretation will be useful in the future to test the performance of the system. 

Now, as we are talking about approximations, we need a way to update them with new observed matches. The original method uses a parameter $k$ for the update.

$ \begin{align}
    \nonumber R_{A, n + 1} = R_{A, n} + k \cdot (S_{A, B} - \bar{P}_{A, B})
\end{align} $

## Update problem

The issue with the vanilla implementation is that it has no way of recognizing when the ratings (and implicitly the underlying aproximated probabilities) are too different from the real values. Neither has it the ability to react to the size of this difference, as the update parameter $k$ is assumed constant. Let us run some tests on the number of games to convergence and the distribution of the difference after convergence.

First, we need to agree on what we mean by "convergence". Say we have the distribution for $D_{A, B}(n) = |P_{A, B}(n) - \bar{P}_{A, B}(n)|$. Integrating the match number $n$ into the equation allows both the actual probabilities and the approximations to change through time, which is a healthy assumption as teams can train and hone their abilities. We can then define the convergence $C_{A, B}$ as:

$ \begin{align}
    \nonumber C_{A, B}(N) = \sum_{n=0}^N w(n) \cdot F_{D_{A, B}(n)}^{-1}(1/2)
\end{align} $

It looks scary, but is quite simple. $F_{D_{A, B}(n)}^{-1}(1/2)$ denotes the median (or $50\%$ quantile) of $D_{A, B}(n)$. We do this because we want something stable on which to measure convergence. The next step is to measure the rate of change of this median as we step further into the games, thus we wight the median according to $n$. We want to tune $w(n)$ such that it gives more importance to bigger $n$'s, which we assume would have converged better than smaller ones.

Let us see how this works if $F_{D_{A, B}(n)}^{-1}(1/2) = e^{-n / 100} + 0.2$ and $w(n) = \frac{e^\frac{-5n}{N}}{\sum_{k=0}^N e^\frac{-5k}{N}}$.

In [ ]:
N = 1000
n = np.array(list(range(N)))
median = np.e ** (-n / 100) + 0.2

fig = go.Figure(layout=utils.layout_dict | dict(
    title='Convergence example', 
    legend=dict(x=0.8, bgcolor=utils.background_color), 
    xaxis=dict(title='n')
))

# Plot the median
fig.add_trace(go.Scatter(x=n, y=median, name='Median of the Difference'))

# Plot convergenve value
weights = np.e ** -np.linspace(5, 0, N)
weights /= sum(weights)
convergence = sum(median * weights)
fig.add_trace(go.Scatter(x=n, y=[convergence] * N, name='Convergence Value'))


fig.show()

We can see that our method is a fairly good approximation, but it still needs to be tested against $N$ to see how it behaves. Fortunately, we know that our choice for the median converges to $0.2$, so we can observe how we reach that value for each increasing choice of $N$:

In [ ]:
convergence = []
Ns = list(range(50, 10000, 50))
for N in Ns:
    # Setup median
    n = np.array(list(range(N)))
    median = np.e ** (-n / 100) + 0.2

    # Calculate convergence value
    weights = np.e ** -np.linspace(5, 0, N)
    weights /= sum(weights)
    convergence.append(sum(median * weights))

# Plot results
fig = go.Figure(layout=utils.layout_dict | dict(
    title='Convergence Sensitivity to N', 
    legend=dict(x=0.8, bgcolor=utils.background_color), 
    xaxis=dict(title='N')
))

fig.add_trace(go.Scatter(x=Ns, y=convergence, name='Approximated Convergence Values'))
fig.add_trace(go.Scatter(x=Ns, y=[0.2] * len(Ns), name='Actual Convergence at Limit'))

fig.show()

## Convergence Tests

Let $P_{A, B}(n) = BRW_{0.25, 0, 0.01}(n)$, where $BRW_{\beta, \mu, \sigma}$ is a Bounded Random Walk with initial value $BRW_{\beta, \mu, \sigma}(0) = \beta$, drift $\mu$ and standard deviation $\sigma$. We first need a function to generate the ratings.

In [ ]:
def calculate_elo_2_players(games_results: np.array, k=1):
    # Get shapes
    N = games_results.shape
    Sa = games_results.copy()

    # Calculate Ratings
    Ra = np.zeros(N)
    pa = np.zeros(N)
    pa[0] += 1/2
    for i in range(1, N[0]):
        # Update rating
        Ra[i] = Ra[i-1] + k * (Sa[i - 1] - pa[i - 1])

        # Update Approx Probabilities
        pa[i] = 1 / (1 + 10 ** (-2 * Ra[i] / 400))

    return Ra, pa

And see how $D_{A, B}(n)$ behaves under this test.

In [ ]:
# Example for k=1
N = [500, 500]
p = utils.non_stationary_bounded_random_walk(initial=1/4, size=N)
games_results = (np.random.random(N) <= p).astype(int)

Ra, pa = calculate_elo_2_players(games_results)

diff = abs(p - pa)
quantiles = np.quantile(diff, [0.1, 0.5, 0.9], axis=1)

# Calculate convergence value
weights = np.e ** -np.linspace(5, 0, N[0])
weights /= sum(weights)
converge_value = sum(quantiles[1] * weights)

# Plot example
x_range = list(range(N[0])) + list(reversed(range(N[0])))

fig = go.Figure(layout=utils.layout_dict | dict(title='Example of D(n) for k=1'))

fig.add_trace(go.Scatter(
        x = x_range, y=list(quantiles[2]) + list(quantiles[0][::-1]), fill='toself', mode='none',
        hoveron='points', fillcolor='lightblue', name=f'80% CI', opacity=0.5
    ))
fig.add_trace(go.Scatter(y=quantiles[1], line=dict(color='lightblue'), name='Median'))
fig.add_trace(go.Scatter(y=[converge_value] * N[0], line=dict(color='red'), name='Convergence Value'))

fig.show()

We can see the output is quite noisy. This means that we need to increase the size of $N$, meaning both the number of games $N[0]$ and the number of parallel iterations $N[1]$. Let us investigate how the convergence values change with different $N$ and $k$ to choose the correct setup for the test.

In [ ]:
def calculate_convergence(N, initial, K):
    # Run games
    p = utils.non_stationary_bounded_random_walk(initial = initial, size=N)
    games_results = (np.random.random(N) <= p).astype(int)

    convergence_values = []
    for k in K:
        # Approximated probabilities
        _, pa = calculate_elo_2_players(games_results, k=k)

        # Calculate difference and median
        diff = abs(p - pa)
        median = np.quantile(diff, 0.5, axis=1)

        # Calculate convergence value
        weights = np.e ** -np.linspace(5, 0, N[0])
        weights /= sum(weights)
        convergence_values.append(sum(median * weights))

    return convergence_values

In [ ]:
K = [1, 2, 5, 7, 10, 20, 50, 100, 1000]
convergence_values = []
N0s = [100, 500, 1000, 2000, 3000, 5000, 10000]
for N0 in N0s:
    N = [N0, 3000]
    convergence_values += [calculate_convergence(N, 1/4, K)]

convergence_values = np.array(convergence_values).T

fig = go.Figure(layout=utils.layout_dict)

for i in range(len(K)):
    fig.add_trace(go.Scatter(x=N0s, y=convergence_values[i], name=f'k = {K[i]}'))

fig.show()

In [ ]:
K = [1, 2, 5, 7, 10, 20, 50, 100, 1000]
convergence_values = []
N1s = [100, 500, 1000, 2000, 3000, 5000, 10000]
for N1 in N1s:
    N = [3000, N1]
    convergence_values += [calculate_convergence(N, 1/4, K)]

convergence_values = np.array(convergence_values).T

fig = go.Figure(layout=utils.layout_dict)

for i in range(len(K)):
    fig.add_trace(go.Scatter(x=N0s, y=convergence_values[i], name=f'k = {K[i]}'))

fig.show()

It appears that $N = [3000, 3000]$ is a good choice for up to $k = 1,000$.

Now for the actual test. It will measure $C_{A, B}$ and the size of the $80\%$ Central Interval (CI) on $D_{A, B}(n)$ for different values of $k$. To test for the convergence value, we will actually look for

$ \begin{align}
    \nonumber Cn_{A, B}(N) = \argmin_n \left[ \left| F_{D_{A, B}(n)}^{-1}(1/2) - C_{A, B} \right| \leq 1\% \right]
\end{align} $

That is, the first game in which the median of $D_{A, B}(n)$ is less than $1\%$ away from the convergence value. This will tell us how many games it took to reach convergence. Let's look at the results.

In [ ]:
# Run games
N = [3000, 3000]
p = utils.non_stationary_bounded_random_walk(initial = 1/4, size=N)
games_results = (np.random.random(N) <= p).astype(int)

time2convergence = []
convergence_values = []
ci = []
K = list(range(1, 11))
K = K + [k * 10 for k in K] #+ [k * 100 for k in K] + [1000]
for k in K:
    # Approximated probabilities
    _, pa = calculate_elo_2_players(games_results, k=k)

    # Calculate difference and median
    diff = abs(p - pa)
    quantiles = np.quantile(diff, [0.1, 0.5, 0.9], axis=1)
    median = quantiles[1]
    central_interval = quantiles[2] - quantiles[0]

    # Calculate convergence value
    weights = np.e ** -np.linspace(5, 0, N[0])
    weights /= sum(weights)
    convergence_value = sum(median * weights)
    first_convergence = np.argmax(abs(median - converge_value) <= 0.01)

    # Fill CI and time to convergence
    time2convergence.append(first_convergence)
    convergence_values.append(convergence_value)
    ci.append(central_interval[first_convergence:].mean())

# Draw a nice plot
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x=K, y=time2convergence, name='Time to Convergence', mode='lines'), secondary_y=False)
fig.add_trace(go.Scatter(x=K, y=convergence_values, name='Convergence Value', mode='lines'), secondary_y=True)
fig.add_trace(go.Scatter(x=K, y=ci, name='Size of 80%CI', mode='lines'), secondary_y=True)

fig.update_layout(**utils.layout_dict)

fig.show()

In [ ]:
# Example for k=20
N = [5000, 1000]
p = utils.non_stationary_bounded_random_walk(initial = 1/4, size=N)
games_results = (np.random.random(N) <= p).astype(int)

Ra, pa = calculate_elo_2_players(games_results, k=20)

diff = abs(p - pa)
quantiles = np.quantile(diff, [0.1, 0.5, 0.9], axis=1)

# Calculate convergence value
weights = np.e ** -np.linspace(5, 0, N[0])
weights /= sum(weights)
converge_value = sum(quantiles[1] * weights)
first_convergence = np.argmax(abs(quantiles[1] - converge_value) <= 0.01)

# Print results
central_interval = quantiles[2] - quantiles[0]
print(f'Time to convergence: {first_convergence}')
print(f'Convergence Value: {converge_value: .2f}')
print(f'80% CI: {central_interval[first_convergence:].mean()}')

# Plot example
l = 1
x = int(N[0] / l)
x_range = list(range(x)) + list(reversed(range(x)))

fig = go.Figure(layout=utils.layout_dict | dict(title='Example of D(n) for k=20'))

fig.add_trace(go.Scatter(
        x = x_range, y=list(quantiles[2][::l]) + list(quantiles[0][::-l]), fill='toself', mode='none',
        hoveron='points', fillcolor='lightblue', name=f'80% CI', opacity=0.5
    ))
fig.add_trace(go.Scatter(y=quantiles[1][::l], line=dict(color='lightblue'), name='Median'))
fig.add_trace(go.Scatter(y=[converge_value] * x, line=dict(color='red'), name='Convergence Value'))

fig.show()

## Improving on Elo updates

To no ones surprise, you either have to choose between fast convergence (a low time to convergence) or a stable convergence (small $80\%$ CI). This is the issue with choosing a static $k$. However, we can tweak $k$ a bit to compensate for this by somehow leveraging our best guess of where we are situated in the convergence process. In simplified terms, we want $k$ to be larger when we have not achieved convergence, and gradually decrease in size until we reach convergence.

We cannot asume that our stochastic process is stationary, thus we cannot state a minimum $k$ for convergence as the variance of the process may change in the future. What we can actually do is perform an approximation of the underlying probabilities, and then vary $k$ depending on how different our current ratings are from that approximation.

### Probability approximation

We will have to put a bit of effort into this approximation, as we will try to approximate probabilities based only on boolean observations. Extra care will be needed so we don't over-estimate the variance of the process and end up having a highly volatile approximation, or not enough adaptability. Let us observe the classical empiric mean formula

$ \begin{align}
    \nonumber &\bar{E}[X] = \sum_{n=1}^{N} \frac{x_n}{N} = \mu
\end{align} $

We can run a little experiment, running this formula on the boolean results of a non-stationary process and observing how it behaves

In [ ]:
N = [1000, 1]

# Simulate probabilities
p = utils.non_stationary_bounded_random_walk(size=N, std=0.002, initial=1/4)
p = p.flatten()

# Simulate Games
games_results = (np.random.random(N[0]) <= p).astype(int)

# Calculate cummulative mean
empiric_mean = games_results.cumsum() / np.linspace(1, N[0], N[0])


# Draw a nice plot
fig = go.Figure(layout=utils.layout_dict | dict(title='Empiric Mean on Non-Stationary process'))

fig.add_trace(go.Scatter(y=p, name='Real Probabilities'))
fig.add_trace(go.Scatter(y=empiric_mean, name='Empiric Mean'))

fig.show()

The empiric formulas are highly sensitive in the first games, and then start losing this sensitivity to become quite flat at the end. Given the ever changing nature of the underlying problem, we need to modify the formulas to give more weight to more recent observations. A simple way of doing this is as follows

$ \begin{align}
    \nonumber &\bar{E}_X(n) = \alpha \cdot \bar{E}_X(n - 1) + (1 - \alpha) \cdot x_n = \bar{\mu}_X(n) 
\end{align} $

We can also initialize the values on $\bar{E}_X(0) = 1/2$. We can call this a step-size mean and denote it by $\bar{E}_{X, \alpha}(0)$.

In [ ]:
N = 1000

# Simulate probabilities
p = utils.non_stationary_bounded_random_walk(size=N, std=0.002, initial=1/4)

# Simulate Games
games_results = (np.random.random(N) <= p).astype(int)

# Calculate cummulative mean and std
alphas = np.array([0.85, 0.875, 0.9, 0.925, 0.95, 0.975, 0.99])
means = []

for i in range(len(alphas)):
    means.append(utils.step_mean(games_results, alphas[i]))


# Draw a nice plot
fig = go.Figure(layout=utils.layout_dict | dict(title='Step-size Mean on Non-Stationary process', legend=dict(x=1)))

fig.add_trace(go.Scatter(y=p, name='Real Probabilities'))

for i in range(len(alphas)):
    fig.add_trace(go.Scatter(y=means[i], name=f'Step-size Mean \u03b1 = {alphas[i]}', opacity=0.2))

fig.show()

Another way to approach the means is by creating a window around our desired value, and computing the mean. This will allow us to compensate for any variability in the procecess and adjust accordinggly. Let us say we have a window of size $w$. Our window mean would be computed as:

$$
    \bar{E}_{X, w}(n) = \sum_{i = \max{(0, n - w)}}^{\min{(N, n + w)}} \frac{x_i}{2 w}
$$

In [ ]:
N = 1000

# Simulate probabilities
p = utils.non_stationary_bounded_random_walk(size=N, std=0.002, initial=1/4)

# Simulate Games
games_results = (np.random.random(N) <= p).astype(int)

# Calculate cummulative mean and std
windows = np.array([5, 10, 20, 50, 100])
means = []

for i in range(len(windows)):
    means.append(utils.apply_rolling_function(games_results, windows[i], np.mean))


# Draw a nice plot
fig = go.Figure(layout=utils.layout_dict | dict(title='Window Mean on Non-Stationary process', legend=dict(x=1)))

fig.add_trace(go.Scatter(y=p, name='Real Probabilities'))

for i in range(len(windows)):
    fig.add_trace(go.Scatter(y=means[i], name=f'Empiric Window w = {windows[i]}', opacity=0.2))

fig.show()

Now, we need a way to combine all the different means into a single output. I trained a simple Linear Regression model in this [notebook](2-teams_model_testing.ipynb) with the following inputs:

- The game number (we are more uncertain in the beginning of the series).
- The boolean results of the games.
- Window mean and stds for $w \in (5, 10, 20, 50, 100)$.
- Step-size mean and stds for $\alpha  \in (0.85, 0.9, 0.95, 0.975, 0.99)$.

This are the results on the same test framework as vanilla Elo updates.    

In [ ]:
def calculate_elo_2_players_improved(games_results: np.array, reg=reg):
    shape = list(reversed(games_results.shape))

    X = utils.generate_X(games_results).T
    pa = reg.predict(X).reshape(shape).T

    return pa

In [ ]:
N = [5000, 1000]
p = utils.non_stationary_bounded_random_walk(initial = 1/4, size=N)
games_results = (np.random.random(N) <= p).astype(int)

pa = calculate_elo_2_players_improved(games_results)

diff = abs(p - pa)
quantiles = np.quantile(diff, [0.1, 0.5, 0.9], axis=1)

# Calculate convergence value
weights = np.e ** -np.linspace(5, 0, N[0])
weights /= sum(weights)
converge_value = sum(quantiles[1] * weights)
first_convergence = np.argmax(abs(quantiles[1] - converge_value) <= 0.01)

# Print results
central_interval = quantiles[2] - quantiles[0]
print(f'Time to convergence: {first_convergence}')
print(f'Convergence Value: {converge_value: .2f}')
print(f'80% CI: {central_interval[first_convergence:].mean()}')

# Plot example
l = 1
x = int(N[0] / l)
x_range = list(range(x)) + list(reversed(range(x)))

fig = go.Figure(layout=utils.layout_dict | dict(title='Improved Convergence Method'))

fig.add_trace(go.Scatter(
        x = x_range, y=list(quantiles[2][::l]) + list(quantiles[0][::-l]), fill='toself', mode='none',
        hoveron='points', fillcolor='lightblue', name=f'80% CI', opacity=0.5
    ))
fig.add_trace(go.Scatter(y=quantiles[1][::l], line=dict(color='lightblue'), name='Median'))
fig.add_trace(go.Scatter(y=[converge_value] * x, line=dict(color='red'), name='Convergence Value'))

fig.show()